# <b>Transformación de los archivos xlxs de la Matriz de requerimientos (DNC) a archivos csv
</b>

Esta libreta convierte archivos Excel del DNC (Detección de Necesidades de Capacitación) a archivos CSV. El objetivo es facilitar el análisis posterior, ya que los archivos originales tienen encabezados y estructuras poco amigables para el procesamiento automático.

Pasos principales:
1. Cargar archivos `.xlsx` de entrada.
2. Limpiar encabezados o columnas innecesarias y guardar los archivos en formato `.csv`.

Esto permite trabajar con los datos de manera más sencilla.


# Importacion de las librerias

In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [610]:
# Ruta de la carpeta donde están los archivos xlxs
carpeta = r"C:\Users\mflor\Downloads\Datos"
Destino=r"C:\Users\mflor\Downloads\Archivoscsv"
# Obtener la lista de archivos csv en la carpeta
archivos= glob.glob(os.path.join(carpeta,"**", "*.xlxs"), recursive=True)
if not os.path.exists(Destino):
    os.makedirs(Destino)


## <b>Limpieza de los archivos </b>

In [620]:
# Leer el archivo Excel, saltando filas iniciales
#df = pd.read_excel(r"C:\Users\mflor\Downloads\MATRIZ DE COMPETENCIAS  DNC - Comercio Exteriorbelen.xlsx", skiprows=6)
archivos=[r"C:\Users\mflor\Downloads\MATRIZ DE COMPETENCIAS  DNC - Comercio Exteriorbelen.xlsx"]
for archivo in archivos:
    # Se lee el archivo quitando los encabezados que no tienen información relevante
    df = pd.read_excel(archivo, skiprows=6, header=None)
    # Eliminar la primera columna
    df = df.iloc[:, 1:]
    # Eliminar el primer, segundo y tercer renglón
    df = df.drop(df.index[[0,3,4]])
    # Reiniciar índice para que los encabezados estén alineados
    df = df.reset_index(drop=True)
    # Eliminar la columna 2 y 3
    df = df.drop(df.columns[[1, 2]], axis=1)
    # Reiniciar las columnas para que los encabezados estén alineados
    df.columns = range(df.shape[1])
    # Encontrar que columna contiene la palabra curso
    curso_idx = df.iloc[0].astype(str).str.strip().str.lower().tolist().index('curso')
    # Copia las celdas a la derecha de "curso"  y las pega en el renglon de abajo
    df.iloc[1, curso_idx:] = df.iloc[0, curso_idx:]
    # Reiniciar índice para que los encabezados estén alineados
    df = df.reset_index(drop=True)
    # Eliminar el primer renglon
    df = df.drop(index=[df.index[0]])
    # Reiniciar índice para que los encabezados estén alineados
    df = df.reset_index(drop=True)
     # Reiniciar índice para que los encabezados estén alineados
    df.columns = range(df.shape[1])
    primer_renglon = df.iloc[0]  # Toma el primer renglón
    # Encuentra las columnas donde el valor no sea nulo ni vacío, esto lo hacemos para encontrar la nomina del trabajador
    columnas_no_vacias = primer_renglon[primer_renglon.notna() & (primer_renglon.astype(str).str.strip() != '')].index.tolist()
    #print("Columnas no vacías en el primer renglón:")
    #print(columnas_no_vacias)
    # Luego, estas nominas se ponen en las dos columnas siguientes que le corresponden al trabajador
    for columna in columnas_no_vacias:
        if columna<curso_idx:
            df.iloc[0,[columna+1,columna+2]]=df.iloc[0,columna]
        else:
            break
    df.iloc[0,0]="Tema"
    # Se elimina las columnas cuyo encabezado es vacío
    df = df.loc[:, ~df.iloc[0].isna()]
    # Se eliminan renglones con información inecesaria
    competencias_idx = df[0].astype(str).str.strip().str.lower().tolist().index('competencias conductuales')
    df = df.iloc[:competencias_idx-1]
    # Finalmente el  archivo limpio se guarda en la carpeta de destino como un archivo csv
    nombre_csv = os.path.splitext(os.path.basename(archivo))[0] + "1.csv"
    df.to_csv(os.path.join(Destino,nombre_csv), index=False, encoding='utf-8-sig',header=None)

A continuación, se muestra el estado de los archivos antes y después del procesamiento.

### Archivo original 
![Archivo Original](https://github.com/MariaBelenFL/MatrizDNC/blob/main/imagenes/DNC.png)

### Archivo después de limpiar y transformar en un archivo csv
![Archivo Limpio](imagenes/DNC2.png)